#                          <center> <div color = "blue">INTELLIGENCE ARTIFICIELLE</div> <br /><br /> PROJET IA for HumanForYou
</center>        





###                             <center> CESICDP GROUPE 2 </center>

#####  <center> Adrian SALAUM <br /><br /> Coralistone METSA<br /><br /> Romain COTTINEAU  <br /><br />Simon BRIFFAUD <br /><br /> </center>


##### ########################################################################################################################

##### ########################################################################################################################

##                     <center> <br />  I. RAPPEL DU CONTEXTE </center>                                                     

L'entreprise de produits pharmaceutiques HumanForYou basée en Inde emploie environ 4000 personnes. Cependant, chaque année elle subit un turn-over d'environ 15% de ses employés nécessitant de retrouver des profils similaires sur le marché de l'emploi.

La direction trouve que ce niveau de turn-over n'est pas bon pour l'entreprise car :

Les projets sur lesquels étaient les employés quittant la société prennent du retard ce qui nuit à la réputation de l'entreprise auprès de ses clients et partenaires.

Un service de ressources humaines de taille conséquente doit être conservé car il faut avoir les moyens de trouver les nouvelles recrues.

Du temps est perdu à l'arrivée des nouveaux employés car ils doivent très souvent être formés et ont besoin de temps pour devenir pleinement opérationnels dans leur nouvel environnement.

Le direction fait donc appel à vous, spécialistes de l'analyse de données, pour déterminer les facteurs ayant le plus d'influence sur ce taux de turn-over et lui proposer des modèles afin d'avoir des pistes d'amélioration pour donner à leurs employés l'envie de rester.

Données fournies
Un certain nombre de données concernant les employés vous a donc été transmis par le service des ressources humaines.

Il s'agit de fichiers textes au format CSV.

Les données ont été anonymisées : un employé de l'entreprise sera représenté par le même EmployeeID dans l'ensemble des fichiers qui suivent.

##               <center>  <br /> <br /> II. Pre-traitement des données et analyse exploratoire </center>   

Voici quelques étapes à suivre pour pré-traiter les données pour une utilisation avec K-means :

1. Intégration des données : Intégrez les données des différentes sources en une seule source de données. Si vous travaillez avec des données provenant de plusieurs sources, vous devez les intégrer en une seule source de données. Cela vous permettra de traiter les données comme une seule entité et de les analyser plus facilement.

1. Nettoyer les données : Assurez-vous que vos données sont propres et ne contiennent pas de valeurs manquantes, de doublons ou d'autres anomalies. Si nécessaire, supprimez ou remplacez les données manquantes.

2. Normaliser les données : Normalisez les données en les mettant à l'échelle de sorte que chaque variable ait une plage de valeurs comparable. La normalisation peut être effectuée en utilisant la méthode de la moyenne et de l'écart type, la méthode de la plage ou la méthode de la normalisation de l'amplitude.

3. Réduire la dimensionnalité : Si vous travaillez avec des données à haute dimensionnalité, utilisez des techniques de réduction de dimensionnalité telles que l'analyse en composantes principales (PCA) pour réduire la dimensionnalité de vos données et faciliter leur analyse.

4. Identifier les valeurs aberrantes : Les valeurs aberrantes peuvent fausser les résultats de K-means, il est donc important de les identifier et de les traiter correctement. Les valeurs aberrantes peuvent être supprimées ou remplacées par des valeurs plus appropriées.

5. Sélectionner les caractéristiques : Si vous travaillez avec des données qui contiennent de nombreuses caractéristiques, il peut être judicieux de sélectionner les caractéristiques les plus pertinentes pour votre analyse.

En résumé, le pré-traitement des données pour K-means comprend le nettoyage des données, la normalisation des données, la réduction de la dimensionnalité, l'identification et le traitement des valeurs aberrantes et la sélection des caractéristiques. En effectuant ces étapes, vous pouvez améliorer la qualité de vos données et obtenir des résultats plus significatifs à l'aide de K-means.

##  <center>  <br /> <br /> 1. Chargement des données </center>
Puisque les données proviennent d'un fichier CSV, nous utiliserons la fonction read_csv de Pandas pour charger les données dans un DataFrame.

### Ca c'est avant de charger les données.

In [23]:
import os
import tarfile
from six.moves import urllib
import pandas as pd

DOWNLOAD_URLS = [
    "https://raw.githubusercontent.com/ArkaniosCesi/IA_project/main/employee_survey_data.csv?token=GHSAT0AAAAAAB766XSKAB6XUE4OXLOCMOI2ZAPJZTQ",
    "https://raw.githubusercontent.com/ArkaniosCesi/IA_project/main/general_data.csv?token=GHSAT0AAAAAAB766XSL3YGUVN7PEMCF7YPEZAPJZUA",
    "https://raw.githubusercontent.com/ArkaniosCesi/IA_project/main/in_time.csv?token=GHSAT0AAAAAAB766XSKGYHR2E3COLIVKN7KZAPKC2Q",
    "https://raw.githubusercontent.com/ArkaniosCesi/IA_project/main/manager_survey_data.csv?token=GHSAT0AAAAAAB766XSK5H7XTJQDK6PMXFCKZAPKDGA",
    "https://raw.githubusercontent.com/ArkaniosCesi/IA_project/main/out_time.csv?token=GHSAT0AAAAAAB766XSLU2NDXSVVONIMICCGZAPKDQQ"
]
HOUSING_PATH = os.path.join("datasets", "employees")

def load_project_data(download_urls=DOWNLOAD_URLS, housing_path=HOUSING_PATH):    
    dataframes = []
    
    for url in DOWNLOAD_URLS:
        # Télécharge les fichiers d'archive depuis les URL
        df = pd.read_csv(url, index_col=0)
        dataframes.append(df)
    
    return dataframes    

 ### Vérifier si les données sont correctement chargées

In [24]:
data = load_project_data()

5


In [28]:
for df in data:
    print(df.head())
    print(df.info())
    print('\n'*5)

            EnvironmentSatisfaction  JobSatisfaction  WorkLifeBalance
EmployeeID                                                           
1                               3.0              4.0              2.0
2                               3.0              2.0              4.0
3                               2.0              2.0              1.0
4                               4.0              4.0              3.0
5                               4.0              1.0              3.0
<class 'pandas.core.frame.DataFrame'>
Int64Index: 4410 entries, 1 to 4410
Data columns (total 3 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   EnvironmentSatisfaction  4385 non-null   float64
 1   JobSatisfaction          4390 non-null   float64
 2   WorkLifeBalance          4372 non-null   float64
dtypes: float64(3)
memory usage: 137.8 KB
None






    Attrition     BusinessTravel              Department  DistanceFromHome  \
Age

### Données de la dernière évaluation du manager

##               <center>  <br /> <br />  2. Création du jeu de test </center>   

##  <center>  <br /> <br /> 3. Étude des données </center>

##  <center>  <br /> <br /> 4. Analyse des données </center>

## <center>  <br /> <br /> 5. Nettoyer les données </center>

##  <center>  <br /> <br /> III. Choix de (des) l'algorithme(s) d'"IA" retenu(s) </center>


## justification des choix et les traitements effectués pour faire face aux problématiques classiques 


## <center>  <br /> <br /> IV. Analyse des résultats obtenus et leur interprétation </center>

##  <center>  <br /> <br /> V. démarche mise en œuvre pour améliorer un modèle  </center>

### <center>  <br /> <br />  1. les différents modèles effectués . </center>

### <center>  <br /> <br />  1.1   K-means . </center>

### <center>  <br /> <br />  1.2  Validation croisée . </center>

### <center>  <br /> <br />  1.3  Les autres algorithmes qu'on va tester. </center>

### <center>  <br /> <br />  2. choix du modèle retenu au final parmi les différents essais effectués en justifiant pas rapport au besoin du client et des métriques. </center>



### <center>  <br /> <br /> les propositions finales justifiées </center>

## <center>  <br /> <br /> VI. RESSOURCES </center>  

## <center>  <br /> <br /> VIII. LEXIQUE </center>  